In [21]:
# extension that autoformats all code to PEP8
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [22]:
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from sklearn.linear_model import ElasticNet
from IPython.display import display
import plotly.express as px

<IPython.core.display.Javascript object>

In [23]:
# Using updated dataframe from preprocessing notebook
df = pd.read_csv("../data/listings.csv")
display(df)

,Unnamed: 0,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,...,neighbourhood_Williamsbridge,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,0,149,1,9,0.21,6,365,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,225,1,45,0.38,2,355,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,2,89,1,270,4.64,1,194,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,3,80,10,9,0.10,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,4,200,3,74,0.59,1,129,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38832,38832,129,1,1,1.00,1,147,0,0,1,...,0,0,0,0,0,0,0,0,1,0
38833,38833,45,1,1,1.00,6,339,0,0,0,...,0,0,0,0,0,0,0,0,1,0
38834,38834,235,1,1,1.00,1,87,0,0,0,...,0,0,0,0,0,0,0,0,1,0
38835,38835,100,1,2,2.00,1,40,1,0,0,...,0,0,0,0,0,0,0,1,0,0


<IPython.core.display.Javascript object>

$$
1 / (2 * n_{samples}) * ||y - Xw||^2_2
+ alpha * l_1 ratio * ||w||_1
+ 0.5 * alpha * (1 - l_1 ratio) * ||w||^2_2
$$ 
or 
$$
a * ||w||_1 + 0.5 * b * ||w||_2^2
$$

In [24]:
df = df.drop(
    [
        "Unnamed: 0"
        #         "name",
        #         "host_id",
        #         "neighbourhood_group",
        #         "neighbourhood",
        #         "latitude",
        #         "longitude",
        #         "room_type",
        #         "last_review",
    ],
    axis=1,
)
display(df)

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,...,neighbourhood_Williamsbridge,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,149,1,9,0.21,6,365,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,225,1,45,0.38,2,355,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,89,1,270,4.64,1,194,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,80,10,9,0.10,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,200,3,74,0.59,1,129,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38832,129,1,1,1.00,1,147,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
38833,45,1,1,1.00,6,339,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
38834,235,1,1,1.00,1,87,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
38835,100,1,2,2.00,1,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


<IPython.core.display.Javascript object>

In [25]:
# Create training data, holding out 10% of data for testing
from sklearn.model_selection import train_test_split  # Returns pandas dataframe

Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    df.drop("price", axis=1), df["price"], test_size=0.2, random_state=42
)
# Ytrain = Xtrain["price"]
# Xtrain, Xtest = Xtrain.drop("price", axis=1), Xtest.drop("price", axis=1)

display(Xtrain)
display(Xtest)
display(Ytrain)

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsbridge,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
6263,3,153,2.90,1,238,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
18571,2,76,2.78,2,270,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
31589,3,2,0.38,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
25228,1,35,2.18,1,356,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
32717,1,28,4.57,1,111,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,4,119,2.23,1,273,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11284,3,7,0.21,13,74,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
38158,1,5,5.00,1,124,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
860,3,25,0.29,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0


,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsbridge,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
8022,2,18,0.37,1,280,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3875,6,1,0.02,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
18829,1,15,0.62,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
32406,2,13,1.83,2,97,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
26597,30,2,0.25,6,255,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,5,80,1.16,2,298,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
14280,2,44,1.33,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11098,1,1,0.02,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
17025,1,4,0.14,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


6263       80
18571     120
31589    1115
25228     120
32717      31
         ... 
6265      150
11284     189
38158     275
860       160
15795      95
Name: price, Length: 31069, dtype: int64

<IPython.core.display.Javascript object>

In [26]:
enm = ElasticNet(alpha=0.01, l1_ratio=0.99)
enm.fit(Xtrain, Ytrain)

C:\Users\Gianni\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.602e+07, tolerance: 1.279e+05
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.01, l1_ratio=0.99)

<IPython.core.display.Javascript object>

In [27]:
# Lets take the coefficients of training and the names of our columns and put them into a dataframe
variables = Xtrain.columns.tolist()
model_coef = pd.DataFrame({"coef": enm.coef_, "names": variables})


# show all the coef that are non zero in descending order
display(model_coef[model_coef["coef"] != 0].sort_values(by="coef", ascending=False))


,coef,names
205,236.477815,neighbourhood_Tribeca
182,148.838442,neighbourhood_Sea Gate
228,94.830045,room_type_Entire home/apt
7,77.875374,neighbourhood_group_Manhattan
85,72.663886,neighbourhood_Flatiron District
...,...,...
206,-33.574848,neighbourhood_Two Bridges
103,-43.943079,neighbourhood_Harlem
139,-44.456802,neighbourhood_Morningside Heights
214,-68.188482,neighbourhood_Washington Heights


<IPython.core.display.Javascript object>

In [28]:
# Now lets predict with our testing data
predictions = enm.predict(Xtest)

# Compared to the actual values
pred_v_act = pd.DataFrame({"predictions": enm.predict(Xtest), "actual": Ytest})
display(pred_v_act)

from sklearn.metrics import *

score = r2_score(Ytest, predictions)
print("The r-squared value is ", score)
print("The adjusted r-squared value is ", 1 - (((1 - score) * (len(pred_v_act.index) - 1))/(len(pred_v_act) - len(pred_v_act.columns) - 1)))


,predictions,actual
8022,129.161889,90
3875,178.502001,150
18829,177.101990,145
32406,133.917824,120
26597,181.802853,62
...,...,...
2923,77.086484,56
14280,33.412338,60
11098,114.594042,60
17025,208.911892,97


The r-squared value is  0.20645724598986248
The adjusted r-squared value is  0.2062528563558612


<IPython.core.display.Javascript object>